## (1) 메일 헤더 파싱

In [1]:
import os
import pandas as pd

In [8]:
#!pip install eml_parser

In [2]:
import datetime
import eml_parser

In [3]:
### json 형태(디렉토리(객체) 또는 리스트(배열))로된 값을 DataFrame 형태로 변환 ###
# 디렉토리와 리스트가 아닌 string 또는 datetime 타입의 값이 나올때까지 검사
# string 또는 datetime값이 나오면 해당 key와 value를 DataFrame으로 바꾼 후 리턴
# 계속해서 열이 추가되도록 concat 사용
def toDataFrame(idx, key, value, column):
    col = column + '/' + key
    eml_df = pd.DataFrame()
    
    if type(value) == type(dict()):
        for k in value:
            eml_df = pd.concat([eml_df, toDataFrame(idx, k, value[k], col)], axis=1)
            
    elif type(value) == type(list()):
        # received의 개수를 last_num 열에 추가
        if col == '/header/received':
            num = len(value)
            eml_df = pd.concat([eml_df, pd.DataFrame(data={'last_num':num}, index=[idx])], axis=1)

        v = ''
        for i, d in enumerate(value):
            if type(d) == type(dict()):
                eml_df = pd.concat([eml_df, toDataFrame(idx, str(i), d, col)], axis=1)
                
            else:   # str, email.headerregistry._ContentTypeHeader, ...
                v += ' ' + d
        eml_df = pd.concat([eml_df, pd.DataFrame(data={col: v}, index=[idx])], axis=1)
    
    else:     #str, datetime.datetime, ...
        eml_df = pd.DataFrame(data={col: value}, index=[idx])
        
    return eml_df

In [4]:
### 인자로 받은 line이 필드(필드 이름과 필드 값으로 구성된 것)인지 확인
# 참고자료: http://cr.yp.to/immhf/field.html
def isField(line):
    field = line.split(':')
    answer = True
    # 시간 관련 열의 경우 ':' 2번 이상 등장, ex. 'Sat, 16 May 2020 07:50:25 +0500'
    if len(field) >= 2:     
        fieldName = field[0]
        for word in fieldName:
            if ord(word) < 33 or ord(word) > 126:
                answer = False
    else:
        answer = False
    
    return answer

In [ ]:
### eml 파일의 content 부분 파싱
# 해당 eml 파일의 header content-type 값에 따라 파싱한다.
# header content-type 값이 multipart인 경우 boundary 값과 빈 행('\n')을 이용해 파싱
# header content-type 값이 multipart가 아닌 경우 빈 행('\n')을 이용해 파싱
def content_parsing(idx, header_content_type, file_path):
    f = open(file_path, 'r', encoding='utf-8' )
    contents = {}
    if 'multipart' in header_content_type:
        i = -1
        isbody_content_type = False
        iscontent = False
        # content-type에서 boundary 파라미터 추출
        boundary = header_content_type.split('boundary=')
        boundary = boundary[1][1:-1]
        
        while True:
            line = f.readline()
            if not line:
                print('not line으로 종료')
                break
            
            # 각 content의 시작 부분마다 --boundary 가 존재
            # 맨 마지막에 --boundary--로 끝남
            if line == '--'+boundary+'\n':
                #isbody_content_type = True
                iscontent = True
                i += 1
                contents['content_'+str(i)] = ''
                
            elif line == '--'+boundary+'--\n':
                break
                
#             elif isbody_content_type == True:
#                 # (1) 빈 행 인지 검사하는 것 만으로도 충분할 것 같음
#                 if line == '\n':
#                 #if '\n' == line or not (line[0]==' ' or line[0]=='\t' or isFiled(line)):

#                     iscontent = True
#                     isbody_content_type = False
                
            elif iscontent == True:
                # 간혹 빈 행 다음에 필드 값이 들어있는 경우가 있음
                # ex. 0b2b8c9c-1862-47a4-8747-01884c7e85.eml
                # 메일 내용인데 단순히 필드형식과 같은 경우와 진짜 필드 값인 경우 구분해야 함 ##### 수정 필요 #####
                # 일단 빈 행 다음에 오는 필드 값은 eml-parser에서도 헤더로 파싱 안해줌
                #if isField(line) == True:
                #    continue
                contents['content_'+str(i)] += line
    else:
        iscontent = False
        contents['content_0'] = ''
        while True:
            line = f.readline()
            if not line:
                break
            ###########
            # http://cr.yp.to/immhf/header.html
            # 헤더와 content를 구분하는 기준
            # 헤더 행이 될 수없는 첫 번째 행에서 헤더 읽기를 중지하십시오. 
            # (1) 빈 행 또는 (2) 공백, 탭 또는 필드 이름 과 콜론으로 시작하지 않는 행 . 
            # 이 전략을 권장합니다. 822 호환 메시지와 거의 모든 깨진 메시지를 올바르게 처리합니다.
            ###########
            # (1) 빈 행 인지 검사하는 것 만으로도 충분할 것 같음
            #if '\n' == line:
            if '\n' == line or (line[0]!=' ' and line[0] !='\t' and isField(line)==False):
                iscontent = True
            if iscontent == True:
                contents['content_0'] += line
                
    return pd.DataFrame(data=contents, index=[idx])

In [15]:
### 인자로 받은 디렉토리(folder)에 있는 파일들 중 선택된 파일들에 대해 파싱한 후 DataFrame으로 저장 ###
def parsing(folder):
    file_list = os.listdir(folder)
    
    emls_df = pd.DataFrame()    
    count = 0
    
    for i, file in enumerate(file_list):
        # 원하는 파일이 아니면 제외(원하는 파일 선택)
        # 생각보다 오래걸릴 수 있어서 file[0:2] == '01' 과 같이 더 나눠서 반복해서 실행시키는 것 추천
        if not (file[0:3] == '0b2'):  # 각자 시작하는 16진수 선택
            continue
        
        if count%100 == 0:     # 진행 상태 확인(100개 단위로 file 이름 print)
            print('file name:', file)
        count += 1
        
        #### eml-parsing ####
        # eml-parsing을 이용해서 eml 파일의 header와 body(html(X)) 부분을 파싱 (메일 header & body 파싱)
        file_path = os.path.join(folder, file)
        with open(file_path, 'rb') as fhdl:
            raw_email = fhdl.read()

        ep = eml_parser.EmlParser()
        # error가 발생해도 나머지 파일들에 대한 파싱값 저장하기 위해 try-except 사용
        try:
            parsed_eml = ep.decode_email_bytes(raw_email)   # 파싱한 결과가 담긴 변수(parsed_eml), 디렉토리 형태
        
        except Exception as e:    # 모든 예외의 에러 메시지를 출력할 때는 Exception을 사용
            print('=== Exception:', e, '(during eml_parser) ===')
            print('error file:', file)
            
        #### to DataFrame ####
        # parsed_eml에 들어있는 파싱된 정보들을 pandas dataframe으로 저장하기
        # toDataFrame 함수 이용
        eml_df = pd.DataFrame(data={'file_name':file}, index=[i])
        if type(parsed_eml['body']) != type(list()):
            print('body type ', type(parsed_eml['body']))
        else:
            eml_df = pd.concat([eml_df, toDataFrame(i, 'body', parsed_eml['body'], '')], axis=1)
# 당분간은 header만 분석
            
        if type(parsed_eml['header']) != type(dict()):
            print('header type: ', type(parsed_eml['header']))
        else:
            # 이전의 eml_df와 병합, eml_df에 header와 관련된 열들 추가
            eml_df = pd.concat([eml_df, toDataFrame(i, 'header', parsed_eml['header'], '')], axis=1)
            
        #### content-parsing ####
        # eml 파일의 content 부분을 파싱하여 DataFrame 'content_n'열에 저장(하나의 파일에 content 여러개 있을 수 있음)
        # content-parsing 함수 이용
        # 이전의 eml_df와 병합, eml_df에 'html'열 추가
#         if '/header/header/content-type' in eml_df.columns:
#             eml_df = pd.concat([eml_df, content_parsing(i, eml_df.loc[i,'/header/header/content-type'], file_path)], axis=1)
#         else:
#             print(file, '/header/header/content-type 열이 없음')
# 당분간은 header만 분석
        
        #### file_size ####
        file_size = {'file_size': os.path.getsize(file_path)}
        eml_df = pd.concat([eml_df, pd.DataFrame(data=file_size, index = [i])], axis=1)

    
        # error가 발생해도 나머지 파일들에 대한 파싱값 저장하기 위해 try-except 사용
        try:
            emls_df = pd.concat([emls_df, eml_df])  
        except ValueError as e:
            # 하나의 dataframe에 중복된 열이름이 있을 경우 concat할 때 error 발생
            # 열이름 확인을 위해 eml_df의 열 print
            print('=== ValueError:', e, '(during concat) ===') # error 문구, 마음대로 변경 가능, e변수에 error 내용 저장
            print('error file name:', file)
            print(parsed_eml)
            for c in eml_df.columns:
                print(c)
            print()
        except AssertionError as e:
            print('=== AssertionError:', e, '(during concat) ===') # error 문구, 마음대로 변경 가능, e변수에 error 내용 저장
            print('error file name:', file)
            
    return emls_df

In [16]:
eml_df = parsing('../../dataset/eml') # 파싱할 eml 파일들이 있는 디렉토리 경로입력

file name: 0b200e46-d0f8-4219-8d9d-2266ad6c194d.eml
file name: 0b2a4eae-00af-4a05-bba7-4cc2fe2e12eb.eml


In [17]:
eml_df.to_csv('../../parsed_eml_1.csv', encoding='utf-8-sig') # 결과로 나오는 csv파일을 저장할 디렉토리 경로 입력

In [7]:
for col in eml_df.columns:
    print(col)

file_name
/header/subject
/header/from
/header/to
/header/date
last_num
/header/received/0/src
/header/received/0/from
/header/received/0/by
/header/received/0/with
/header/received/0/date
/header/received/1/src
/header/received/1/from
/header/received/1/by
/header/received/1/for
/header/received/1/date
/header/received
/header/received_domain
/header/received_ip
/header/received_foremail
/header/header/received
/header/header/reply-to
/header/header/mail_from
/header/header/message-id
/header/header/content-type
/header/header/org_rcpt_to
/header/header/x-mailer
/header/header/date
/header/header/x-spam-type
/header/header/x-received-ip
/header/header/from
/header/header/rcpt_to
/header/header/to
/header/header/subject
/header/header/mime-version
/header/header/x-helo
/header/header/x-priority
/header/header/x-mail-from
/header/received/1/with
/header/received_email
/header/header/list-unsubscribe
/header/header/domainkey-signature
/header/header/dkim-signature
/header/received/2/src
